# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

OperationalError: FATAL:  database "sparkifydb" does not exist


In [ ]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True, convert_dates=False,dtype=False)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,None,,None,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = list(df.loc[0, ['song_id','title','artist_id','year','duration']].values)
song_data = [a.item() if type(a) != str else a for a in song_data]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = list(df.loc[0, ['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']])
artist_data

['ARNTLGG11E2835DDB9', 'Clp', '', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath = log_files[0]

In [13]:
df = pd.read_json(filepath, lines=True, convert_dates=False, dtype=False)
df['t'] = pd.datetime(1970,1,1) + pd.to_timedelta(df['ts'], unit='ms')
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,t
0,Mitch Ryder & The Detroit Wheels,Logged In,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Jenny Take A Ride (LP Version),200,1543363215796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:00:15.796
1,The Spill Canvas,Logged In,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,The TIde (LP Version),200,1543363420796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:03:40.796
2,Mogwai,Logged In,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Two Rights Make One Wrong,200,1543363778796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:09:38.796
3,Spor,Logged In,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Way Of The Samurai,200,1543364349796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:19:09.796
4,DJ Dizzy,Logged In,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Sexy Bitch,200,1543364729796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:25:29.796


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 19 columns):
artist           363 non-null object
auth             433 non-null object
firstName        411 non-null object
gender           411 non-null object
itemInSession    433 non-null int64
lastName         411 non-null object
length           363 non-null float64
level            433 non-null object
location         411 non-null object
method           433 non-null object
page             433 non-null object
registration     411 non-null float64
sessionId        433 non-null int64
song             363 non-null object
status           433 non-null int64
ts               433 non-null int64
userAgent        411 non-null object
userId           433 non-null object
t                433 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(4), object(12)
memory usage: 64.4+ KB


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df = df.loc[df['page'] == 'NextSong',:]  # 433 -> 363
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,t
0,Mitch Ryder & The Detroit Wheels,Logged In,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Jenny Take A Ride (LP Version),200,1543363215796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:00:15.796
1,The Spill Canvas,Logged In,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,The TIde (LP Version),200,1543363420796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:03:40.796
2,Mogwai,Logged In,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Two Rights Make One Wrong,200,1543363778796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:09:38.796
3,Spor,Logged In,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Way Of The Samurai,200,1543364349796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:19:09.796
4,DJ Dizzy,Logged In,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Sexy Bitch,200,1543364729796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:25:29.796


In [16]:
t = pd.datetime(1970,1,1) + pd.to_timedelta(df['ts'], unit='ms')
t.head()

0   2018-11-28 00:00:15.796
1   2018-11-28 00:03:40.796
2   2018-11-28 00:09:38.796
3   2018-11-28 00:19:09.796
4   2018-11-28 00:25:29.796
Name: ts, dtype: datetime64[ns]

In [17]:
time_data = [[i,i.hour,i.day,i.weekofyear,i.month,i.year,i.weekday()] for i in t]
column_labels = ('timestamp', 'hour','day','weekofyear','month','year','weekday')

In [18]:
time_df = pd.DataFrame(time_data, columns=column_labels).drop_duplicates()

In [19]:
time_df.shape

(361, 7)

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:
user_df = df[['userId','firstName','lastName','gender','level']].drop_duplicates().astype({'userId':int})
user_df.head()
# given the result, userID here should be integer

,userId,firstName,lastName,gender,level
0,80,Tegan,Levine,F,paid
13,92,Ryann,Smith,F,free
15,74,Braden,Parker,M,free
17,55,Martin,Johnson,M,free
18,40,Tucker,Garrison,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [31]:
df.head(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,t
0,Mitch Ryder & The Detroit Wheels,Logged In,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Jenny Take A Ride (LP Version),200,1543363215796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:00:15.796
1,The Spill Canvas,Logged In,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,The TIde (LP Version),200,1543363420796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:03:40.796
2,Mogwai,Logged In,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Two Rights Make One Wrong,200,1543363778796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:09:38.796
3,Spor,Logged In,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Way Of The Samurai,200,1543364349796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:19:09.796
4,DJ Dizzy,Logged In,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Sexy Bitch,200,1543364729796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:25:29.796
5,Erik Hassle,Logged In,Tegan,F,70,Levine,183.43138,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Hurtful,200,1543364950796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-28 00:29:10.796
13,Phoenix,Logged In,Ryann,F,2,Smith,207.15057,free,"Palestine, TX",PUT,NextSong,1.540685e+12,964,Holdin' On Together,200,1543369249796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-28 01:40:49.796
14,Arctic Monkeys,Logged In,Ryann,F,3,Smith,175.43791,free,"Palestine, TX",PUT,NextSong,1.540685e+12,964,Mardy Bum,200,1543369456796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-28 01:44:16.796
15,tobyMac,Logged In,Braden,M,0,Parker,229.69424,free,"Youngstown-Warren-Boardman, OH-PA",PUT,NextSong,1.540999e+12,801,Burn For You,200,1543377306796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",74,2018-11-28 03:55:06.796
17,Jadakiss / Swizz Beatz / OJ Da Juiceman,Logged In,Martin,M,1,Johnson,191.84281,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,996,Who's Real,200,1543382018796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55,2018-11-28 05:13:38.796


In [30]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    
    timestamp = pd.datetime(1970,1,1) + pd.to_timedelta(row.ts, unit='ms')
    # insert songplay record
    songplay_data = (timestamp, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [28]:
cur.execute('select * from songplays')
cur.fetchmany(5)

[(1,
  datetime.datetime(2018, 11, 28, 0, 0, 15, 796000),
  80,
  'paid',
  None,
  None,
  992,
  'Portland-South Portland, ME',
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'),
 (2,
  datetime.datetime(2018, 11, 28, 0, 3, 40, 796000),
  80,
  'paid',
  None,
  None,
  992,
  'Portland-South Portland, ME',
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'),
 (3,
  datetime.datetime(2018, 11, 28, 0, 9, 38, 796000),
  80,
  'paid',
  None,
  None,
  992,
  'Portland-South Portland, ME',
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'),
 (4,
  datetime.datetime(2018, 11, 28, 0, 19, 9, 796000),
  80,
  'paid',
  None,
  None,
  992,
  'Portland-South Portland, ME',
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chr

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [34]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.